# PySnobal 25 year comparison

Aim to run a longer time series to evaluate PySnobal against the original Snobal.

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from inicheck.tools import MasterConfig, cast_all_variables, get_user_config

import pysnobal
from pysnobal.pysnobal import PySnobal

def nse(obs, model):
    num = np.sum((model - obs)**2)
    denom = np.sum((obs - np.mean(obs))**2)
    return 1 - num/denom

In [ ]:
# Load the historic data and split it up into years

out_dir = '../pysnobal/tests/test_data_point/gold_csv/'

data = pd.read_csv(
    os.path.join(out_dir, 'gold.snobal.data.input.csv'),
    index_col='date_time', parse_dates=True)

# split into water years
wys = data.index.year.unique()[1:]

In [ ]:
# Output to individual files
for wy in wys:
    start_date = pd.to_datetime('10-01-{} 00:00'.format(wy-1))
    end_date = pd.to_datetime('09-30-{} 23:00'.format(wy))

    wy_data = data.loc[start_date:end_date, :]

    wy_data.to_csv(os.path.join(out_dir, '25year', 'wy{}.input.csv'.format(wy)))

In [ ]:
# run PySnobal
config_file = os.path.join(out_dir, 'config.ini')

master_config = os.path.join(Path(pysnobal.__file__).parent, 'pysnobal_core_config.ini')
mcfg = MasterConfig(path=master_config)
config = get_user_config(config_file, mcfg=mcfg)


for wy in wys:
    print(wy)

    # Update the config
    config.raw_cfg['time'].update({
        'start_date': '10-01-{} 00:00'.format(wy-1),
        'end_date': '09-30-{} 23:00'.format(wy)
    })
    config.raw_cfg['files'].update({
        'input_csv': os.path.join('25year', 'wy{}.input.csv'.format(wy)),
        'output_csv': os.path.join('25year', 'wy{}.output.csv'.format(wy))
    })

    config.apply_recipes()
    config = cast_all_variables(config, config.mcfg)

    # run with updated config
    status = PySnobal(config).run()

In [ ]:
# read in the Snobal runs
file_name = '../pysnobal/tests/test_data_point/gold_ipw/gold.snobal.out.25year'
output_label = np.array(['time_s','R_n','H','L_v_E','G','M','delta_Q','G_0','delta_Q_0',
            'cc_s_0','cc_s_l','cc_s','E_s','melt','ro_predict','z_s_0','z_s_l',
            'z_s','rho','m_s_0','m_s_l','m_s','h2o','T_s_0','T_s_l','T_s'])

snobal = pd.read_csv(
    file_name,
    delimiter=" ", names=output_label)
start_date = pd.to_datetime('10-01-1983 00:00')
tdelta = pd.to_timedelta(snobal.time_s, unit='hour')
snobal['date_time'] = start_date + tdelta
snobal['date_time'] = snobal['date_time'].dt.round('min') # Snobal uses decimals for minutes but pandas will convert with microseconds
snobal.set_index('date_time', inplace=True)
snobal.index = snobal.index.tz_localize('MST')
snobal.drop('time_s', axis=1, inplace=True)


In [ ]:
# difference between the models

df = None
for idx,wy in enumerate(wys):

    start_date = pd.to_datetime('10-01-{} 01:00'.format(wy-1))
    end_date = pd.to_datetime('09-30-{} 23:00'.format(wy))
    gold = snobal.loc[start_date:end_date, :]

    file_name = os.path.join(out_dir, '25year', 'wy{}.output.csv'.format(wy))
    new = pd.read_csv(file_name, index_col='date_time', parse_dates=True)

    d = gold - new
    dd = d.describe()

    for col in gold.columns:
        dd.loc['NSE', col] = nse(new[col], gold[col])

    dd['water_year'] = wy
    dd.reset_index(inplace=True)
    dd.rename(columns={'index': 'metric'}, inplace=True)

    if df is None:
        df = dd
    else:
        df = pd.concat([df, dd])
    

In [ ]:
df.set_index(['water_year', 'metric'], inplace=True)

df

In [ ]:
# look at the differences between the models
variable = 'm_s'

fig, ax = plt.subplots(len(wys), 1, figsize=(15, 5*len(wys)))

for idx,wy in enumerate(wys):

    start_date = pd.to_datetime('10-01-{} 01:00'.format(wy-1))
    end_date = pd.to_datetime('09-30-{} 23:00'.format(wy))
    gold = snobal.loc[start_date:end_date, :]

    file_name = os.path.join(out_dir, '25year', 'wy{}.output.csv'.format(wy))
    new = pd.read_csv(file_name, index_col='date_time', parse_dates=True)

    par1 = ax[idx].twinx()
    d = gold[variable] - new[variable]
    par1.plot(new.index, np.zeros_like(new.index), 'k--')
    par1.plot(d.index, d, 'k')

    ax[idx].plot(new.index, new[variable], '-')
    ax[idx].plot(gold.index, gold[variable], '-')
    ax[idx].set_title(wy)
    ax[idx].legend(['pysnobal', 'snobal'])

